In [2]:
import pandas as pd
import numpy as np
train_varients=pd.read_csv("training_variants")
test_varients=pd.read_csv("test_variants")
train_text=pd.read_csv("training_text",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)
test_text=pd.read_csv("test_text",sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)

In [3]:
train = pd.merge(train_varients, train_text, on='ID')
print train.head()

   ID    Gene             Variation  Class  \
0   0  FAM58A  Truncating Mutations      1   
1   1     CBL                 W802*      2   
2   2     CBL                 Q249E      2   
3   3     CBL                 N454D      3   
4   4     CBL                 L399V      4   

                                                TEXT  
0  Cyclin-dependent kinases (CDKs) regulate a var...  
1   Abstract Background  Non-small cell lung canc...  
2   Abstract Background  Non-small cell lung canc...  
3  Recent evidence has demonstrated that acquired...  
4  Oncogenic mutations in the monomeric Casitas B...  


In [4]:
# creating a new column called Text Count
train.loc[:, 'Text_count']  = train["TEXT"].apply(lambda x: len(x.split()))
train.head()

,ID,Gene,Variation,Class,TEXT,Text_count
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...,6089
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...,5722
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...,5722
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...,5572
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...,6202


In [5]:
train[train['Text_count']==1.0]

,ID,Gene,Variation,Class,TEXT,Text_count
1109,1109,FANCA,S1088F,1,null,1
1277,1277,ARID5B,Truncating Mutations,1,null,1
1407,1407,FGFR3,K508M,6,null,1
1639,1639,FLT1,Amplification,6,null,1
2755,2755,BRAF,G596C,7,null,1


In [6]:
dataframe=train[train['Text_count']!=1.0]

In [7]:
len(dataframe[dataframe['Text_count']==1.0])

0

In [8]:
dataframe.head()

,ID,Gene,Variation,Class,TEXT,Text_count
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...,6089
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...,5722
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...,5722
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...,5572
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...,6202


## Mini Bag Of Words

In [19]:
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import string
import matplotlib.pyplot as plt
import collections
def top2000_for_each_class_mini_bag(class_value):
    text=train[train.Class==int(class_value)].TEXT.tolist()
    text1=text
    #this piece of code is removing special character
    for i in range (len(text)):
        for k in text[i].split("\n"):
            text[i]=re.sub(r"[^a-zA-Z0-9]+", ' ', k)
    # removing stop words
    word_dict=[]
    stopword=set(stopwords.words("english"))
    stopword.update((u'fig','1','figure','Figure'))
    stopword.update(str(i) for i in range(2,1000))
    for val in text:
        tokens =[]
        for element in val.split(" "):
            tokens.append(element)
    word_dict = [word for word in tokens if word.lower() not in stopword] 
    counter=collections.Counter(word_dict)
    top_2000_words=counter.most_common(2000)
    top_words=[]
    for i in range(0,len(top_2000_words)):
        top_words.append(top_2000_words[i][0])
    data=[]
    for i in range(len(text1)):
        words=[]
        for k in text1[i].split(" "):
            words.append(k)
        s=""
        for w in words:
            if (w.lower() in top_words):
                s=s+" "+w
        data.append(s)
    return data

In [20]:
def mini_bag():
    temporary_text=[]
    mini_bag=[]
    
    for i in range(1,10):
        temporary_text=top2000_for_each_class_mini_bag(str(i))
        mini_bag.append(temporary_text)
    return mini_bag

In [21]:
mini_bag_of_words=mini_bag()

In [22]:
print mini_bag_of_words[:1]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [24]:
# using count vectorizer of scikit learn bag of words tool
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 
train_data_features = vectorizer.fit_transform(mini_bag_of_words)
print train_data_features.shape

AttributeError: 'list' object has no attribute 'lower'